# Cryptography

In [1]:
# set up
import random
from math import floor, ceil, log2, gcd
import numpy as np

# Part I: Finite prime field

In [2]:
# ===================================
# Globl parameters  
#   p : prime in F_p
# ===================================

def fadd(u,v):   
    return (u + v) % p

def fmul(u,v):
    return (u*v) % p

def fneg(u):
    return (-1*u) % p

# find the inverse of u in F_p, see algorithm from lecture ntoes
def finv(u):
    trace = np.array([[0,0,p,0],[1,0,u,1]])
    i = 1
    while trace[i,2]>0:
        if trace[i,2]==1:
            q = trace[i-1,2]
            r = 0
            t = 0
        if trace[i,2]!=1:
            q = floor(trace[i-1,2]/trace[i,2]) # largest possible value for which q*r(i-1)<=r(i-2)
            r = fadd(trace[i-1,2],fneg(fmul(q,(trace[i,2]))))
            t = fadd(trace[i-1,3],fneg(fmul(q,(trace[i,3]))))
        newrow = np.array([[i+1,q,r,t]])
        trace = np.vstack([trace, newrow])
        i=i+1
        if i > 999:
            print('Algorithm stopped')
            break
    return (trace[i-1,3])
    
def fsub(u,v):
    return fadd(u,fneg(v))

# compute u/v in F_p
def fdiv(u,v):
    return fmul(u,finv(v))
    # should this be return(u // v)

# compute the square root of u mod p
def fsqrt(u):
    def legendre_symbol(u):
        ls = pow(u, (p - 1) // 2, p)
        return -1 if ls == p - 1 else ls
    if legendre_symbol(u) != 1: return 0
    if u     == 0:              return 0
    if p % 4 == 3:              return pow(u, (p + 1) // 4, p)
    s,e  = p-1, 0
    while s % 2 == 0:  s,e  = s // 2, e + 1
    n = 2
    while legendre_symbol(n) != -1:  n += 1
    x,b,g,r = pow(u, (s + 1) // 2, p), pow(u, s, p), pow(n, s, p), e
    while True:
        t,m = b, 0
        for m in range(r):
            if t == 1: break
            t = pow(t, 2, p)
        if m == 0: return x
        gs = pow(g, 2 ** (r - m - 1), p)
        g = (gs * gs) % p
        x = (x * gs) % p
        b = (b * g) % p
        r = m

In [3]:
p = 11

In [4]:
fadd(7,9)

5

In [5]:
fmul(7,9)

8

In [6]:
fneg(7)

4

In [7]:
finv(7)

8

In [8]:
fsub(7,9)

9

In [9]:
fdiv(7,9)

2

In [10]:
fsqrt(3)

5

In [11]:
fsqrt(2)    # No squre-root of 2 modulo 11, should return 0

0

# Part II: Elliptic Curve Group

In [12]:
# ===================================
# Globl parameters
#   a,b in y^2 = x^3 + a x + b
# ===================================

# We will use 0 as our identity element
def gop(A,B):
    #!!! Fill in using finite field operations (such as fadd, fmul, fneg, finv, fsub, fdiv)  
    if A == 0: 
        return B   # infinity is represented as 0.
    elif B == 0:
        return A
    elif A[0] == B[0] and A[1] == fneg(B[1]):
        return 0
    elif A == B:
        m = fdiv(fadd(fmul(3, fmul(A[0],A[0])), a), fmul(2,A[1]))
        x = fsub(fmul(m,m),fmul(2,A[0]))
        y = fsub(0,fadd(fmul(m,fsub(x,A[0])),A[1]))
        C = [x,y]
        return (C)
    else:
        m = fdiv(fsub(B[1],A[1]),fsub(B[0],A[0]))
        x = fsub(fsub(fmul(m,m),A[0]),B[0])
        y = fsub(0,fadd(A[1],fmul(fsub(x,A[0]),m)))
        C = [x,y]
        return (C)

# compute the inverse of A in the group
def ginv(A):
    x = A[0]
    y = fneg(A[1])
    C = [x,y]
    return (C)
    #!!! Fill in using finite field operations  

    
# compute A^K in the group, see algorithm in lecture notes
def gpow(A,k):
    if k == 0:
        return(0)
    elif k<0:
        return(gpow(ginv(A),-k))
    elif k == 1:
        return(A)
    elif k % 2 == 0:
        r = gpow(A,k/2)
        return(gop(r,r))
    elif k % 2 == 1:
        r = gpow(A,(k-1)/2)
        return(gop(gop(r,r),A))
    else:
        print('Operation failed')
        
    #!!! Fill in using the group operation gop

In [13]:
A = [0,1]
B = [0,2]
p = 3
A[1] == fneg(B[1])

True

In [14]:
a,b,p = 1,2,7

gop([3,2],[0,4]) # your program should return [6, 0]

[6, 0]

In [15]:
a,b,p = 1,2,7

ginv([3,2])

[3, 5]

In [16]:
a,b,p = 1,2,7

gpow([3,2],5)

[0, 3]

In [17]:
a,b,p = 1,2,7
S     = [[0, 3], [0, 4], [1, 2], [1, 5], [3, 2], [3, 5], [4, 0], [6, 0], 0]
e     = len(S)

a,b,p = 0,1,5
S     = [[0,1], [0,4], [2,2], [2,3], [4,0], 0]
e     = len(S)

print('Multiplication Table for a = %d,  b = %d,  p = %d'%(a,b,p))
print()
print('%-10s' %'', end=" ")
for j in range(e):
    print('%-8s' %S[j], end=" ")
print()
print()
#print('\n----------------------------------------------------------------------')
for i in range(e-1):
    print('%-10s' %S[i], end=" ")
    for j in range(e):
        s = gop(S[i],S[j])
        print('%-8s' %s, end=" ")
    print()

Multiplication Table for a = 0,  b = 1,  p = 5

           [0, 1]   [0, 4]   [2, 2]   [2, 3]   [4, 0]   0        

[0, 1]     [0, 4]   0        [2, 3]   [4, 0]   [2, 2]   [0, 1]   
[0, 4]     0        [0, 1]   [4, 0]   [2, 2]   [2, 3]   [0, 4]   
[2, 2]     [2, 3]   [4, 0]   [0, 4]   0        [0, 1]   [2, 2]   
[2, 3]     [4, 0]   [2, 2]   0        [0, 1]   [0, 4]   [2, 3]   
[4, 0]     [2, 2]   [2, 3]   [0, 1]   [0, 4]   0        [4, 0]   


# Part III: Encode & Decode  
(You don't need to do anything about this part)

In [18]:
# ===================================
# Globl parameters
#   a,b : in y^2 = x^3 + ax + b 
#   p   : prime in F_p
#   w   : encoding parameter
# ===================================

# break a text into smaller blocks
def split(text):
    n = floor(log2(p/w)) // 8 - 1
    messages = [text[i:i+n] for i in range(0, len(text), n)]
    return messages

def encode(message):
    x = int.from_bytes(message.encode(), byteorder='little')
    for xp in range(w*x,w*(x+1)):
        c = (xp**3 + a*xp + b) % p
        yp = fsqrt(c)
        if yp != 0: return [xp,yp]   
    print("Very unlikely thing happened!!! You need to increase the value of w.")
    return FAIL;   

def decode(m):
    x = m[0]//w
    length = ceil(x.bit_length() / 8)
    return x.to_bytes(length, byteorder='little').decode()

In [19]:
a = -3
b = 2455155546008945908945579827824778998937758030093070285253
p = 6277101735386680763835789423207666416083908700390324961279
w = 2**8

text = "The urge to discover secrets is deeply ingrained in human nature"
print('=====================================================================')
print(text)
print('=====================================================================')

messages = split(text)
for message in messages:
    print(message)
    m = encode(message)
    print(m)
    message = decode(m)
    print(message)
    print('--------------------------------------------')

The urge to discover secrets is deeply ingrained in human nature
The urge to discover s
[11026951511822505740824816598564065624702472009904837633, 6151605368174235746863821334283590061124563105620907872397]
The urge to discover s
--------------------------------------------
ecrets is deeply ingra
[9333557845285353305487404376077288563174454565137507585, 258006309802765647444669041990289651379883185299850026340]
ecrets is deeply ingra
--------------------------------------------
ined in human nature
[148265109643662281674581825977957840452429567387904, 1875102528059364570960654373176424825105735193767258147086]
ined in human nature
--------------------------------------------


# Part IV: ElGamal based on Elliptic Group over finite prime field

In [20]:
def init_receiver():   # Bob does the following once.
    # Choose a group G (parametrized by a,b,p) and  an encoding/decoding scheme (parametrized by w)
    a = -3
    b = 2455155546008945908945579827824778998937758030093070285253
    p = 6277101735386680763835789423207666416083908700390324961279
    w  = 2**8
    group = [a,b,p,w]

    # Choose a private key k > 0
    k = 1309500873854749099831268309457391717722850279
    private_key = k
    
    # Construct a public key g,h
    g = [602046843592854700404561654781648738821600320709046855698,
         1580573966983949753944652941399918970627913726427538969730]
    h = gpow(g,k)
    public_key = [g,h]
    
    return group, public_key, private_key 

    
def send(text):   # Alice sends messages to Bob.
    messages  = split(text)
    encrypted = []
    for message in messages:
        m  = encode(message)
        s  = random.randrange(p//(2*w), p//w)
        c0 = gpow(g,s)
        c1 = gop(gpow(h,s),m)
        encrypted.append([c0,c1])
    return encrypted


def receive(encrypted): # Bob receives messages from Alice.
    text = ""
    for c in range(len(encrypted)):
        c0, c1 = encrypted[c]
        m = gop(gpow(ginv(c0),k),c1)
        message = decode(m)
        text = text + message
    return text

In [21]:
group, public_key, private_key = init_receiver()

a,b,p,w = group         # Broadcast to the whole world
g,h     = public_key    # Broadcast to the whole world
k       = private_key   # Keep it secret

text = "The urge to discover secrets is deeply ingrained in human nature; even the least curious mind is roused by the promise of sharing knowledge withheld from others. Some are fortunate enough to find a job which consists of the solution of mysteries, but most of us are driven to sublimate this urge by the solving of artificial puzzles designed for our entertainment. Detective stories or crossword puzzles cater for the majority; the solution of secret codes may be the pursuit of the few. (John Chadwick from The Decipherment of Linear B, as quoted in The Code Book, by Simon Singh)"
print('\n== Text ================================================================')
print(text)

encrypted = send(text)
print('\n== Encrypted text =====================================================')
print(encrypted)

decrypted = receive(encrypted)
print('\n== Decrypted text =====================================================')
print(decrypted)

print('\n== Compare  ===========================================================')
if text == decrypted: 
    print("Same!")
else: 
    print("Different!  Debug the program :-)")


== Text ================================================================
The urge to discover secrets is deeply ingrained in human nature; even the least curious mind is roused by the promise of sharing knowledge withheld from others. Some are fortunate enough to find a job which consists of the solution of mysteries, but most of us are driven to sublimate this urge by the solving of artificial puzzles designed for our entertainment. Detective stories or crossword puzzles cater for the majority; the solution of secret codes may be the pursuit of the few. (John Chadwick from The Decipherment of Linear B, as quoted in The Code Book, by Simon Singh)

== Encrypted text =====================================================
[[[6059599081574750903784518617223211671601606894243494974043, 4253077601211698607461421543774941314549680482849078117716], [1642468258979806719927101593029097875372396055454936998318, 5339934909634610198639835792312798104720446503952573476831]], [[4790635300153761836574